In [1]:
!pip install yargy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=8722ba26fd8e5b1b5c1387a4f5ef1446d582228073f63fb68b43fade4d6e31a7
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path='/content/drive/MyDrive/NLP/task1/news.txt'

In [4]:
def read_texts(fn: str):
    texts=[]
    ans=[]
    with open(fn, "rt", encoding="utf-8") as f:
        for line in f:
            texts.append(line.strip().split("\t")[2])
            ans.append(line.strip().split("\t")[0])
    return texts, ans

texts, ans = read_texts(path)

In [5]:
import nltk
from nltk.corpus import stopwords

import re
import numpy as np
import pandas as pd

import gensim
from yargy.tokenizer import MorphTokenizer
from collections import Counter

tokenizer = MorphTokenizer()

nltk.download("stopwords")
ru_stopwords = set(stopwords.words("russian"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
def normalize_text(text: str) -> str:
    tokens = [
        tok.normalized for tok in tokenizer(text) if tok.normalized not in ru_stopwords
    ]
    return " ".join(tokens)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer

vectorizer = TfidfVectorizer(max_df=0.2, min_df=10)
X = vectorizer.fit_transform([normalize_text(text) for text in texts]).toarray()

In [51]:
from sklearn.decomposition import PCA

pca = PCA(n_components=500)
X = pca.fit_transform(X)

In [52]:
def transform_text(text: str) -> np.ndarray:
    normalized_text = normalize_text(text)
    vect = vectorizer.transform([normalized_text]).toarray()
    return pca.transform(vect)

In [53]:
ans=np.array(ans)

In [54]:
def text_avg(text):
  summ=0
  for word in text:
    summ+=word
  return summ/len(text)

X_avg=[]
for text in X:
  X_avg.append(text_avg(text))

In [55]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_avg, ans, test_size=0.2)

svc = SVC()
X_train = np.array(X_train).reshape(-1, 1)
svc.fit(X_train, y_train)

SVC()

Находить средний вектор плохая идея


In [56]:
get_pred=transform_text("Футбольное Динамо забило гол и выиграло").reshape(-1, 1)
get_pred3=text_avg(get_pred).reshape(-1, 1)
svc.predict(get_pred3)

array(['economics'], dtype='<U9')

In [57]:
from sklearn.metrics import accuracy_score

X_test=np.array(X_test).reshape(-1, 1)
y_pred = svc.predict(X_test.reshape(-1, 1))
accuracy = accuracy_score(y_test, y_pred)
print(f"Точность модели: {accuracy}")

Точность модели: 0.18


Наиболее точных резульатат из прещедщих мне в голову идей дает простое отправление полных векторов текста на обучение


In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, ans, test_size=0.2)

svc2 = SVC()
svc2.fit(X_train, y_train)

SVC()

In [59]:
y_pred = svc2.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Точность модели: {accuracy}")

Точность модели: 0.8705


Попробуем брать первую 10 самых больших векторов и классифицировать по ним


In [74]:

def text_top10_id(text):
  text_copy=text.copy()
  text_copy.sort()
  top10=[]
  for i in range(10):
    for j in range(len(text)):
      if text_copy[i]==text[j]:
        #top10.append(j)
        top10.append(text[j])
        break
  return top10

In [75]:
X_top10=[]
for text in X:
  X_top10.append(text_top10_id(text))

X_train, X_test, y_train, y_test = train_test_split(X_top10, ans, test_size=0.2)

svc3 = SVC()
#X_train = np.array(X_train).reshape(-1, 1)
svc3.fit(X_train, y_train)

SVC()

Точность возросла, но не сильно


In [76]:
#X_test=np.array(X_test).reshape(-1, 1)
y_pred = svc3.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Точность модели: {accuracy}")

Точность модели: 0.2765
